In [4]:
from dotenv import load_dotenv
import requests, os
import pandas as pd
load_dotenv()

api_key = os.getenv('riot_api_key')
player_url = os.getenv('url')
player_puuid = os.getenv('puuid_url')
match_url = os.getenv('match_url')
rank = os.getenv('tft_rank')

In [5]:
#raw data from api.

def fetch_matches():
    gameName = input('Type in your in-game name: ')
    tagLine = input('Type in your tagline: ')

    player_url_edited = player_url.format(gameName=gameName, tagLine=tagLine, api_key=api_key)

    response_puuid = requests.get(player_url_edited)
    
    if response_puuid.status_code == 200:
        puuid = response_puuid.json()['puuid']
    else:
        print(f'Error : {response_puuid.status_code}')
        return None

    player_tft = player_puuid.format(puuid=puuid, api_key=api_key)

    response_tft = requests.get(player_tft)

    if response_tft.status_code == 200:
        player_data = response_tft.json()
    else:
        print(f'error: {response_tft.status_code}')
        return None


    match_url_edited = match_url.format(puuid=puuid, api_key=api_key)
    response_match = requests.get(match_url_edited)

    if response_match.status_code == 200:
        match_id = response_match.json()
    else:
        print(f'error: {response_match.status_code}')
        return None
    
    rank_edited = rank.format(puuid=puuid, api_key=api_key)

    response_rank = requests.get(rank_edited)

    if response_rank.status_code == 200:
        rank_data = response_rank.json()
    else:
        print(f'error: {response_tft.status_code}')
        return None

    all_matches = []

    match_details = os.getenv('match_dets')

    for i in match_id:
        match_formated = match_details.format(match_id=i, api_key=api_key)
        match_formated = requests.get(match_formated)
        
        if match_formated.status_code == 200:
            match_data = match_formated.json()
            all_matches.append(match_data)


    return all_matches, puuid, gameName, rank_data # SE VOCE VAI RETORNAR QUATRO COISAS AQUI, TEM QUE CAPTURAR DENOVO NA CELL ABAIXO!!!!



In [6]:


matches, my_puuid, gameName, rank_data = fetch_matches() # TIPO ASSIM!!!!!


def get_details(matches, my_puuid):
    details = []

    for match in matches:
        for player in match['info']['participants']:
            if player['puuid'] == my_puuid:
                
                placements = player['placement']
                companion = player['companion']

                details.append({
                    'placement': player['placement'],
                    'companion': player['companion'],
                    })
                break

    return details

details = get_details(matches, my_puuid)
df = pd.DataFrame(details)
print(df)

    

   placement                                          companion
0          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
1          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
2          8  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
3          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
4          2  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
5          8  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
6          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
7          1  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
8          3  {'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...
9          3  {'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...
